In [6]:
import pandas as pd
from classification import TextClassification

# Sentiment Classification 

In [7]:
sent_df= pd.read_csv('data/Amazon_Unlocked_Mobile.csv')
sent_df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [8]:
# Call the class 
cl = TextClassification(sent_df, 'Reviews')
sent_df_encoded = cl.encode_positive_rating('Rating')

sent_df_encoded.head()

Sample Size 413840


/Users/yutine/Documents/Computer Science/Python/nlp_sentiment_classification/classification.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['Positively Rated'] = np.where(self.df['Rating'] > 3, 1, 0)


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1


In [9]:
# Split data into training and test sets
pos, non_pos = cl.spliting_train_testing_binary_class(sent_df_encoded, 'Positively Rated')

Sample size of Class 1 data: 230669; Percentage: 74.83%
Sample size of Class 0 data: 77601; Percentage: 25.17%


### Evaluate Models

In [10]:
cl.logistic_regressions_count_vect()
cl.logistic_regression_Tfidf()

/Users/yutine/Documents/Computer Science/Python/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC, Logistic Regressions using Count Vectorizer 0.9755280488173645
Smallest coefficients: ['worst', 'junk', 'garbage', 'unusable', 'useless', 'false', 'waste', 'terrible', 'scam', 'worthless'] 
Largest coefficients: ['excelente', 'excelent', 'exelente', 'loves', 'loving', 'excellent', 'perfecto', 'complaints', 'bien', 'wonderful']
Number of features: 18024
AUC, Logistic Regressions w/TF-IDF Vectorizer 0.9819569872950711


# Spam Classification

## Data Review & Train /Test Spliting

In [ ]:
df = pd.read_csv('data/review_w_spam_label.csv')

# Apply one-hot encoding to spam data 
df['target'] = np.where(df['target']=='spam',1,0)
spam = df[df['target'] == 1]

print(f'{(len(spam)/len(df))*100} percent of data is spam.')
df.head(10)

### Call class and function to split training /testing data 

In [ ]:
SentClass = TextClassification(df, 'text', 'target')

## Model Evaluation 
- 1. Multinomial **Naive Bayes classifier** model w/ Count Vectorizer
- 2. Multinomial **Naive Bayes classifier** model w/ TFI-DF Vectorizer (Minimum Document Frequency = **3**)
- 3. **SVM** w/ TF-IDF Vectorizer and document length as additional feature (Minimum Document Frequency = **5**)
- 4. **Logistic Regression** model w/ Tfidf Vectorizer (Minimum Document Frequency = **5** ; and using **word n-grams from n=1 to n=3** unigrams, bigrams, and trigrams).
  Add following additional features:
    - the length of document (number of characters)
    - **number of digits per document**
- 5. **Logistic Regression** model w/ Count Vectorizer 
    - First 2000 rows only of training data X_train 
    - Ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**
    - Using this document-term matrix and the following additional features:
      * the length of document (number of characters)
      * number of digits per document
      * **number of non-word characters (anything other than a letter, digit or underscore.)**
    - fit a Logistic Regression model with regularization C=100 and max_iter=1000

In [ ]:
# AUC, multinomial Naive Bayes classifier model w/ Count Vectorizer
count_vect = SentClass.NB_classifier_count_vectorizer()

# AUC, multinomial Naive Bayes classifier model w/ TFI-DF Vectorizer
tfidf_vect, tfidf_X_train_vectorized = SentClass.get_tfidf_vectorizer(3)
SentClass.NB_classfier_tfidf(tfidf_vect, tfidf_X_train_vectorized)

# AUC, SVM using TF-IDF and document length as additional feature
SentClass.Tfidf_Vector_with_SVM(5)

# AUC, Logistic Regression using TF-IDF and document length + Num of digits per document as additional features 
SentClass.logistic_regression_Tfidf_ngrams()

count_vect_ngrams, LR_model_w_count = SentClass.logistic_regressions_count_vect_ngrams(2000)

## Feature Review
### TFI-DF Features 
- Print out features with the smallest and largest tf-idf, among the **max** tf-idf values

In [ ]:
SentClass.get_tfidf_features(tfidf_vect, 20, 20)

### Review non-word characters in dataset 
- Non-word characters: anything other than a letter, digit or underscore
- Use RegEx `\w` and `\W` character classes
- Calculate the average number of non-word characters per document for not spam and spam documents

In [ ]:
nonword_in_spam, nonword_in_nonspam = SentClass.calculate_avg_nonword_char('text')

In [ ]:
# Review the non-word characters found in spam 
nonword_in_spam.head(10)

### find the 10 smallest and 10 largest coefficients from the model

- The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.


In [ ]:
SentClass.return_coef(count_vect_ngrams, LR_model_w_count)